In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
a0 = np.zeros((2,3,7), dtype=float) #Polarization, K={10logG_P, H_P, beta_P}, sumover


a0[0,0,:] = [-0.333199e+02,
            -0.484707e+00,
            0.209419e-01,
            -0.722253e-03,
            -0.197359e-05,
            0.554852e-06,
            -0.412505e-08]

a0[1,0,:] = [-0.352134e+02,
            -0.700816e+00,
            0.218222e-01,
            -0.428971e-03,
            -0.101374e-04,
            0.357085e-06,
            0.317954e-08]

a0[0,1,:] = [0.206369e+01,
             0.150857e-01,
            -0.484725e-03,
             0.282771e-04,
            -0.130800e-05,
            -0.208157e-07,
             0.149365e-08]

a0[1,1,:] = [ 0.171875e+01,
            -0.215342e-02,
            -0.495723e-03,
             0.303353e-04,
            -0.237352e-05,
            -0.249487e-07,
             0.283361e-08]


a0[0,2,:] =  [-0.252960e-01,
            -0.828636e-03,
            -0.127067e-03,
            0.223988e-05,
            0.209310e-06,
            -0.628579e-09,
            -0.177410e-09]

a0[1,2,:] =  [0.263735e-01,
            0.333145e-02,
            -0.920586e-04,
            -0.452963e-05,
            0.117252e-06,
            0.528832e-08,
            -0.989594e-10]


a2 = np.zeros((2,3,4), dtype = float)

a2[0,:,0]= [-0.310929e-02,
           0.844082e-03,
           -0.294355e-04]

a2[0,:,1]= [0.569782e-03,
           -0.817576e-04,
           0.250476e-05]

a2[0,:,2]= [-0.134223e-04,
            0.176263e-05,
           -0.530116e-07]

a2[0,:,3]= [0.974349e-07,
           -0.122837e-07,
           0.367063e-09]


a2[1,:, 0] = [-0.820475e-03, 
             0.585086e-03, 
             -0.217526e-04]

a2[1,:, 1] = [0.228444e-03, 
             -0.433336e-04, 
             0.137112e-05]

a2[1,:, 2] = [-0.279677e-05, 
             0.570146e-06, 
             -0.174164e-07]

a2[1,:, 3] = [-0.186039e-07, 
             0.426204e-09, 
             -0.738717e-11]

a1V = np.zeros(4,3)

a1V[:,0] =[0.219575e+00,
            -0.514068e-02,
            0.443523e-05,
            -0.875588e-05]

a1V[:,0] = [0.588161e-03,
            0.518480e-03,
            -0.104422e-04,
            -0.153 837e-06]

a1V[:,2] = [-0.355617e-03,
            0.656609e-05,
            0.119783e-05,
            -0.353267e-07]


a1H = np.zeros(4,4)

a1H[:,0] = [0.442604e+00,
            -0.825727e-02,
            0.168152e-03,
            -0.212250e-04]

a1H[:,1] = [0.128898e-01,
            0.982790e-03,
            -0.136218e-03,
            0.33 8862e-05]

a1H[:,2]= [-0.112481e-02,
            0.100865e-03,
            -0.854751e-05,
            0.287316e-06]

a1H[:,3] = [-0.332487e-04,
            0.207059e-05,
            -0.133341e-06,
            0.515220e-08]

In [9]:
def getA0P(wspd, a0, theta):
    ny, nx = wspd.shape
    X = np.zeros((2,3,ny, nx), dtype=np.float64)
    for i in range(2): # vertical polarization is number index 0
        for j in range(3):
            val = 0
            for k in range(7):
                val += a0[i,j,k] * (theta[:,:]-40)**k
            X[i,j,:,:] = val
    G = np.exp(X[:,0,:,:]/10)
    H = X[:,1,:,:]
    beta = X[:,2,:,:]
    A0 = G*wspd^(H)*np.exp(beta* wspd)
    return A0


def getA2P(wspd, a2, theta):
    ny, nx = wspd.shape
    A2 = np.zeros((2, ny, nx), dtype=np.float64)
    for i in range(2):
        for n in range(4):
            for m in range(3):
                A2[i,:,:] += a2[i,m,n] * wspd**(m+1) * (theta-10)**(n+1) 
    return A2


def getA1V(wspd, a1V, theta):
    ny, nx = wspd.shape
    A1V = np.zeros((2, ny, nx), dtype=np.float64)
    for m in range(4):
        for n in range(3):
            A1V += a1V[m,n]*wspd**m * (theta - 40)**n

    return A1V

def getA1H(wspd, a1V, theta):
    ny, nx = wspd.shape
    A1H = np.zeros((2, ny, nx), dtype=np.float64)
    for m in range(4):
        for n in range(4):
            A1H += a1H[m,n]*wspd**m * (theta - 40)**n

    return A1H



def getSigma(wpsd, theta, phi):
    A0 = getA0P(wspd, a0, theta)
    A2 = getA2P(wspd, a2, theta)
    A1V = getA1V(wspd, a1V, theta)
    A1H = getA1H(wspd, a1V, theta)
    A1 = np.stack((A1V, A1H), axis=0)

    sigma = A0*(1+A1*np.cos(np.deg2rad(phi)) + A1 * np.cos(2*np.deg2rad(phi)))

    return sigma
            
    